## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-05-17-07-34 +0000,nyt,Files Reveal Epstein’s Money Mingled with Sili...,https://www.nytimes.com/2026/02/05/business/ep...
1,2026-02-05-17-06-48 +0000,nypost,Mamdani health dept. ‘global oppression workin...,https://nypost.com/2026/02/05/us-news/zohran-m...
2,2026-02-05-17-05-51 +0000,bbc,Remaining worshippers kidnapped in Nigeria chu...,https://www.bbc.com/news/articles/c4g3v4pjwgjo...
3,2026-02-05-17-05-44 +0000,nypost,Son of ex-NFL player Kevin Johnson breaks sile...,https://nypost.com/2026/02/05/us-news/ex-nfler...
4,2026-02-05-17-00-00 +0000,wsj,The Utah Mom Who Got the World Hooked on ‘Dirt...,https://www.wsj.com/business/retail/dirty-soda...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2543/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
58,trump,35
2,epstein,27
148,new,22
73,up,13
593,california,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
338,2026-02-04-19-57-31 +0000,nypost,President Trump called Savannah Guthrie to off...,https://nypost.com/2026/02/04/us-news/presiden...,80
260,2026-02-04-23-11-15 +0000,nypost,Trump gets help in his critical minerals race ...,https://nypost.com/2026/02/04/us-news/house-pa...,77
248,2026-02-04-23-57-59 +0000,bbc,Supreme Court allows California to use new vot...,https://www.bbc.com/news/articles/ckgyvn5gv25o...,72
125,2026-02-05-12-07-28 +0000,nyt,Melinda French Gates Addresses New References ...,https://www.nytimes.com/2026/02/04/us/melinda-...,72
209,2026-02-05-03-27-44 +0000,bbc,Trump says America should move on from Epstein...,https://www.bbc.com/news/articles/cy4gj71z0m0o...,71


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
338,80,2026-02-04-19-57-31 +0000,nypost,President Trump called Savannah Guthrie to off...,https://nypost.com/2026/02/04/us-news/presiden...
248,72,2026-02-04-23-57-59 +0000,bbc,Supreme Court allows California to use new vot...,https://www.bbc.com/news/articles/ckgyvn5gv25o...
229,71,2026-02-05-00-56-40 +0000,latimes,L.A. Olympics head Casey Wasserman withstood a...,https://www.latimes.com/california/story/2026-...
151,63,2026-02-05-11-00-00 +0000,nypost,Pro-ICE billboards calling out ‘liberal hypocr...,https://nypost.com/2026/02/05/us-news/pro-ice-...
250,54,2026-02-04-23-48-00 +0000,wsj,Federal Reserve governor Lisa Cook sees a grea...,https://www.wsj.com/economy/central-banking/fe...
238,44,2026-02-05-00-37-28 +0000,nypost,Google parent Alphabet latest tech giant to an...,https://nypost.com/2026/02/04/business/google-...
65,43,2026-02-05-14-47-58 +0000,cbc,"U.S., Russia agree to re-establish military-to...",https://www.cbc.ca/news/world/united-states-ru...
72,38,2026-02-05-14-39-47 +0000,nypost,Rick Caruso considers return to LA mayor’s rac...,https://nypost.com/2026/02/05/us-news/rick-car...
378,38,2026-02-04-17-43-52 +0000,nypost,Second person dies during Japan skiing trip — ...,https://nypost.com/2026/02/04/world-news/secon...
289,36,2026-02-04-22-17-00 +0000,wsj,Washington Post Lays Off One-Third of Staff in...,https://www.wsj.com/business/media/washington-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
